# Convolutional Neural Networks (CNNs) & Its Architecture

You'll be using a popular benchmark dataset - Fashion-MNIST - in order to create, train, and evaluate a convolutional neural network whose Alexnet diagram appears below.

<div style="text-align: center;"> <img src = "res/cnn_fashion/alexnet_initial_cnn.jpg" width="80%"/> </div>

After you create this CNN, you'll then make a variety of modifications to it in order to better understand CNN hyperparameters.

### The Fashion-MNIST Data Set

Before you go ahead and load in the data, it's good to take a look at what you'll exactly be working with! The Fashion-MNIST dataset is a dataset of Zalando's article images, with 28x28 grayscale images of 70,000 fashion products from 10 categories, and 7,000 images per category. 

Fashion-MNIST is similar to the MNIST dataset that you might already know, which you use to classify handwritten digits. That means that the image dimensions, training and test splits are similar to the MNIST dataset.

<div style="text-align: center;"> <img src = "res/cnn_fashion/dataset_cover.png" width="30%"/> </div>

# 0 | Google Colab Setup

In [ ]:
import os
import shutil
import stat

In [ ]:
def copy_safe(src, dst, max_len=200):
    """Copy files, skip long paths"""
    skipped = 0
    for root, dirs, files in os.walk(src):
        rel_path = os.path.relpath(root, src)
        dst_root = os.path.join(dst, rel_path) if rel_path != '.' else dst
        if len(dst_root) < max_len:
            os.makedirs(dst_root, exist_ok=True)
            for file in files:
                dst_file = os.path.join(dst_root, file)
                if len(dst_file) < max_len:
                    try: shutil.copy2(os.path.join(root, file), dst_file)
                    except: skipped += 1
                else: skipped += 1
        else: skipped += len(files)
    return skipped

In [ ]:
# Setup resources if needed
setup_ran = False
if not os.path.exists('res'):
    print("Setting up resources...")
    setup_ran = True
    
    # Cleanup, clone, copy
    repo = 'deep_learning_resources'
    if os.path.exists(repo):
        shutil.rmtree(repo, onerror=lambda f,p,e: os.chmod(p, stat.S_IWRITE) or f(p))
    
    !git clone --depth=1 https://github.com/jjv31/deep_learning_resources
    
    if os.path.exists(f'{repo}/res'):
        skipped = copy_safe(f'{repo}/res', 'res')
        print(f"Setup complete! {'(' + str(skipped) + ' long filenames skipped)' if skipped else ''}")
    
    shutil.rmtree(repo, onerror=lambda f,p,e: os.chmod(p, stat.S_IWRITE) or f(p))

In [ ]:
# Only refresh if we just downloaded resources
if setup_ran:
    from IPython.display import Javascript, display
    import time
    
    print("Refreshing images...")
    
    # Try browser refresh + aggressive image reload
    display(Javascript(f'''
    try {{ setTimeout(() => window.location.reload(true), 2000); }} catch(e) {{}}
    
    const t = {int(time.time())};
    document.querySelectorAll('img').forEach((img, i) => {{
        if (img.src.includes('res/')) {{
            const src = img.src.split('?')[0];
            setTimeout(() => img.src = src + '?v=' + t + '_' + i, i * 50);
        }}
    }});
    '''))
    
    print("If images don't appear, press Ctrl+Shift+R to hard refresh!")
else:
    print("Resources already exist, skipping setup.")

# 1 | Import Dataset

### 1.1 | Imports and Auxilary Methods

Just run these.

In [ ]:
# Datasets & Math
import numpy as np

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical # One Hot Encoding

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

In [ ]:
# Neural networks
from keras.datasets import fashion_mnist
import keras as keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from keras.optimizers import Adam


In [ ]:
# Plots the performance of the neural network
def plot_performance(training_values, validation_values, metric_name = "Recall"):

    epochs = range(1, len(training_values) + 1)
    
    sns.set() 
    plt.plot(epochs, training_values, '-', label=f'Training {metric_name}')
    plt.plot(epochs, validation_values, ':', label=f'Validation {metric_name}')

    plt.title(f'Training and Validation {metric_name}')
    plt.xlabel('Epoch')
    plt.ylabel(metric_name)
    plt.legend(loc='lower right')
    plt.plot()

### 1.2 | Loads Dataset

In [ ]:
# We'll be loading the dataset directly from KERAS, and they have already seperated it into a trianing & testing split.
(train_X,train_Y), (test_X,test_Y) = fashion_mnist.load_data()

In [ ]:
# Note that these are NOT pandas datasets: they're numpy arrays
# Thus, we can't use .head() and other Pandas functions.
type(train_X)

### 1.3 | Inspect Datasets

Let's now analyze how images in the dataset look like. Even though you know the dimension of the images by now, it's still worth the effort to analyze it programmatically. For example, you might have to rescale the image pixels and resize the images.

In [ ]:
print('Training data shape : ', train_X.shape, train_Y.shape) # 60,000 images that are 28 x 28
print('Testing data shape : ', test_X.shape, test_Y.shape) # 10,000 images that are 28 x 28

In [ ]:
# Find the unique numbers from the train labels
classes = np.unique(train_Y)

print('Total number of outputs : ', len(classes))
print('Output classes : ', classes)

### 1.4 | Visualize Dataset Contents

In [ ]:
# We can't visualize these contents normally because they're images.
# Here is how our datast is stored: train_X[image_number, row_number, col_number]
# image_number - the first index of our array. It refers to the image number in our dataset
# row_number - the second index refers to the row of the image
# col_number - the second index refers tot he column number

# Thus, each image in the dataset is a 28 x 28 array with a brightness value between 0 (black) and 255 (white). 

# Let's inspect the first image
train_X[0,:,:]

In [ ]:
# Instead of viewing the raw image data, let's render the image via MatPlotLib

plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121) # For displaying the graphs nicely. refers to rows_in_subplot, cols_in_subplot, position_of_current_graph.
plt.imshow(train_X[0,:,:], cmap='gray')
plt.title(f"Ground Truth : {train_Y[0]}")

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title(f"Ground Truth : {test_Y[0]}")

In [ ]:
# *****************************
# EXERCISE
# ******************************
# Do the same thing except display the 5th image in our training & testing set.

plt.figure(figsize=[5,5])

# Training
plt.subplot(121)
plt.imshow(train_X[0,:,:], cmap='gray')
plt.title(f"Ground Truth : {train_Y[0]}")

# Testing
plt.subplot(122)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title(f"Ground Truth : {test_Y[0]}")

# 2 | Preprocessing


As you could see in the above plot, the images are grayscale images have pixel values that range from 0 to 255. Also, these images have a dimension of 28 x 28. As a result, you'll need to preprocess the data before you feed it into the model.

As a first step, convert each 28 x 28 image of the train and test set into a matrix of size 28 x 28 x 1 which is fed into the network.

### 2.1 | Scaling

In [ ]:
# The brightness values of our image range from 0 to 255
# However, neural networks are best suited for handling values with a smaller rnage, like 0-1.
# Thus, we'll convert these values to a 0-1 decimal.

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

### 2.2 | One-hot encoding

In [ ]:
# This is a mutliclass classification problem with 10 classes.
# Thus, we'll need 10 output neurons: 1 neuron per class.
# To make our datset compatible with our neural network, the class needs to be one-hot encoded.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

### 2.3 | Create Validation Set

We have a lot of data in this dataset. We have so much data that we'll do the "proper" thing of evaluating our neural network's training via a validation set rather than the testing set. This helps ensure that the testing set represents an unbiased view of the model's performance in the real world

<div style="text-align: center;"> <img src = "res/cnn_fashion/dataset_split.jpg" width="30%"/> </div>

Note that we'll be creating our validation set using 10% of the training set (c.f., 10% of the whole dataset!). Thus, the numbers on the diagram aren't exactly right, but the underlying concept is the same.

In [ ]:
train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.1, random_state=42)

In [ ]:
print(f"Images in the training set {train_X.shape[0]}")
print(f"Images in the validation set {valid_X.shape[0]}")
print(f"Images in the testing set {test_X.shape[0]}")

# 3 | Construct CNN

In [ ]:
# We'll output 1 output node per class, consistent with multiclass problems
NUM_OUTPUT_NODES = len(train_Y_one_hot[0])
EPOCHES = 20

### 3.1 | Construct CNN

In [ ]:
# Constructs the initial CNN
initial_cnn = Sequential()

# Input layer
initial_cnn.add( Input( shape= (28,28,1) ) ) 

# Convolution - Max Pooling Layer Pairs
initial_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
initial_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

initial_cnn.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
initial_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

initial_cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
initial_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

# Feedforward Neural Network
initial_cnn.add(Flatten())
initial_cnn.add(Dense(128, activation='relu'))
initial_cnn.add(Dense(NUM_OUTPUT_NODES, activation='softmax'))

In [ ]:
# Compiles & Displays it
initial_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(),  
                    metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")], )

initial_cnn.summary()

In [ ]:
hist = initial_cnn.fit(train_X, train_label, validation_data=(valid_X, valid_label),
                                batch_size=128, epochs=EPOCHES, )

### 3.2 | Evaluate on Testing Set

In [ ]:
# Evaluation on training set
print('Train loss:', hist.history["loss"][-1])
print('Train accuracy:', hist.history["accuracy"][-1])

In [ ]:
# Evaluation on testing set
test_eval = initial_cnn.evaluate(test_X, test_Y_one_hot, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

### 3.3 | Evaluate the CNN's Training

In [ ]:
loss, val_loss = hist.history["loss"], hist.history["val_loss"]
plot_performance(loss, val_loss, "Loss (Error)")

In [ ]:
acc, val_acc = hist.history["accuracy"], hist.history["val_accuracy"]
plot_performance(acc, val_acc, "Accuracy")

# 4 | Your CNNs

### 4.0 | Section Overview

The initial CNN from the previous section is pasted below, in each subsequent section. For each subsection, you'll be asked to modify one/more parameters of the neural network, and in the process, you'll learn more about what each parameter does.

### 4.1 | Task 1: Remove Max Pooling Layers

#### 4.1.1 | Construct CNN

In [ ]:
# *****************************
#  EXERCISE
# *****************************

# Remove the max pooling layers of your CNN. What happens?


# Constructs the initial CNN
your_cnn = Sequential()

# Input layer
your_cnn.add( Input( shape= (28,28,1) ) ) 

# Convolutional Components
your_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

# Feedforward Neural Network
your_cnn.add(Flatten())
your_cnn.add(Dense(128, activation='relu'))
your_cnn.add(Dense(NUM_OUTPUT_NODES, activation='softmax'))

In [ ]:
# Compiles & Displays it
your_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(),  
                    metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")], )

your_cnn.summary()

In [ ]:
your_hist = your_cnn.fit(train_X, train_label, validation_data=(valid_X, valid_label),
                                batch_size=128, epochs=EPOCHES, )

#### 4.1.2 | Evaluates your CNN

In [ ]:
# Evaluation on testing set
your_test_eval = your_cnn.evaluate(test_X, test_Y_one_hot, verbose=0)

print("Your Model: ")
print('Test loss:', your_test_eval[0])
print('Test accuracy:', your_test_eval[1])

print("\nOriginal Model: ")
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
loss, val_loss = your_hist.history["loss"], your_hist.history["val_loss"]
plot_performance(loss, val_loss, "Loss (Error)")

#### 4.1.3 | Your Analysis

In [ ]:
# **********************
# EXERCISE
# ***************************

# How is training time affected? How was performance affected?
# Type your answer in the multi-line string below.

'''

YOUR ANSWER HERE


'''

### 4.2 | Task 2: Add Another Convolutional Layer - Max Pooling Layer Hybrid

#### 4.2.1 | Construct CNN

In [ ]:
# *****************************
#  EXERCISE
# *****************************

# Add another convolutional layer
# Add this AFTER the max pooling layer that appears immediately after the 128-neuron Max Pooling Layer
# Make this however many neurons you want, provided it's less than or equal to 1024.

try:
    # Constructs the initial CNN
    your_cnn = Sequential()

    # Input layer
    your_cnn.add(Input(shape=(28, 28, 1)))

    # Convolutional Component
    your_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu'))
    your_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    your_cnn.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu'))
    your_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    your_cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu'))
    your_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    your_cnn.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu'))
    your_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    # Feedforward Neural Network
    your_cnn.add(Flatten())
    your_cnn.add(Dense(128, activation='relu'))
    your_cnn.add(Dense(NUM_OUTPUT_NODES, activation='softmax'))

except Exception as e:
    print(f"ERROR. Your CNN cannot be constructed or compiled due to the following error: \n{e}")


In [ ]:
# There's no need to train this neural network ;)

In [ ]:
# ***************************
# Exercise 
# ***************************

# What just happened and why?
# Hint: Look at the diagram of the initial CNN that appears at the start of this practicum
# Type your answer below 

'''
YOUR ANSWER HERE

'''

### 4.3 | Task 3: Invert the CNN

#### 4.3.1 | Construct CNN

In [ ]:
# *****************************
#  EXERCISE
# *****************************

# Invert the CNN such that there's more neurons closer to the input image, less neurons further out


# Constructs the initial CNN
your_cnn = Sequential()

# Input layer
your_cnn.add( Input( shape= (28,28,1) ) ) 

# Convolutional Component
your_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

# Feedforward Neural Network
your_cnn.add(Flatten())
your_cnn.add(Dense(128, activation='relu'))
your_cnn.add(Dense(NUM_OUTPUT_NODES, activation='softmax'))

In [ ]:
# Compiles & Displays it
your_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(),  
                    metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")], )

your_cnn.summary()

In [ ]:
your_hist = your_cnn.fit(train_X, train_label, validation_data=(valid_X, valid_label),
                                batch_size=128, epochs=EPOCHES, )

#### 4.3.2 | Evaluate CNN

In [ ]:
# Evaluation on testing set
your_test_eval = your_cnn.evaluate(test_X, test_Y_one_hot, verbose=0)

print("Your Model: ")
print('Test loss:', your_test_eval[0])
print('Test accuracy:', your_test_eval[1])

print("\nOriginal Model: ")
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
loss, val_loss = your_hist.history["loss"], your_hist.history["val_loss"]
plot_performance(loss, val_loss, "Loss (Error)")

#### 4.3.3 | Your Analysis

In [ ]:
# **********************
# EXERCISE
# **********************


# What happened to the model's performance? Did anything else happen?
# Type your answer in the multi-line string below

'''
YOUR ANSWER HERE

'''

### 4.4 | Task 4: Add Dropout

#### 4.4.1 | Construct CNN

In [ ]:
# *****************************
#  EXERCISE
# *****************************

# Dropout layers are extremely common in CNNs because they reduce overfitting without impairing performance.
# Why don't you add a dropout layer after each MaxPooling layer below?
# I did the first for you. 
# Feel free to adjust the decimal within the dropout parameter. It's the percentage of neurons in the previous layer that'll be removed.


# Constructs the initial CNN
your_cnn = Sequential()

# Input layer
your_cnn.add( Input( shape= (28,28,1) ) ) 

# Convolutional Component
your_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )
your_cnn.add( Dropout(0.50) ) # Feel free to adjust 0.25 to any value between 0 and 1.00

your_cnn.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') )
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

# Feedforward Neural Network
your_cnn.add(Flatten())
your_cnn.add(Dense(128, activation='relu'))
your_cnn.add(Dense(NUM_OUTPUT_NODES, activation='softmax'))

In [ ]:
your_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(),  
                    metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")], )

your_cnn.summary()

In [ ]:
your_hist = your_cnn.fit(train_X, train_label, validation_data=(valid_X, valid_label),
                                batch_size=128, epochs=EPOCHES, )

#### 4.4.2 | Evaluate CNN

In [ ]:
# Evaluation on testing set
your_test_eval = your_cnn.evaluate(test_X, test_Y_one_hot, verbose=0)

print("Your Model: ")
print('Test loss:', your_test_eval[0])
print('Test accuracy:', your_test_eval[1])

print("\nOriginal Model: ")
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
loss, val_loss = your_hist.history["loss"], your_hist.history["val_loss"]
plot_performance(loss, val_loss, "Loss (Error)")

#### 4.4.3 | Your Analysis

In [ ]:
# ****************************
# EXERCISE
# ****************************

# What happened to your model's performance?
# Did dropout help or hurt your model's performance on the testing set? Why?

'''
YOUR ANSWER HERE

'''

# 5 | Freestyling

### 5.0 | Section Overview

This section is arrayed just like the individual task sections in §4.0. However, you're free to make whatever changes to the original CNN that you wish. Just make sure your CNN compiles!

### 5.1 | Construct CNN

In [ ]:
# Constructs the initial CNN
your_cnn = Sequential()

# Input layer
your_cnn.add( Input( shape= (28,28,1) ) ) 

# Convolutional Component
your_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') ) # Change activation function
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') ) # Change activation function
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

your_cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='leaky_relu') ) # Change activation function
your_cnn.add( MaxPooling2D(pool_size=(2, 2) ) )

# Feedforward Neural Network
your_cnn.add(Flatten())
your_cnn.add(Dense(128, activation='relu')) # Change activation function
your_cnn.add(Dense(NUM_OUTPUT_NODES, activation='softmax'))

In [ ]:
your_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(),  
                    metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")], )

your_cnn.summary()

In [ ]:
your_hist = your_cnn.fit(train_X, train_label, validation_data=(valid_X, valid_label),
                                batch_size=128, epochs=EPOCHES, )

### 5.2 | Evaluate CNN

In [ ]:
# Evaluation on testing set
your_test_eval = your_cnn.evaluate(test_X, test_Y_one_hot, verbose=0)

print("Your Model: ")
print('Test loss:', your_test_eval[0])
print('Test accuracy:', your_test_eval[1])

print("\nOriginal Model: ")
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
loss, val_loss = your_hist.history["loss"], your_hist.history["val_loss"]
plot_performance(loss, val_loss, "Loss (Error)")

# 6 | Detailed CNN Evaluation

### 6.1 | Test Set Performance

Let's retrieve the CNN's performance on the testing set, which was calcualted earlier

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

### 6.2 | Calculate Predicted Classes

In [ ]:
# Let's visualize the images that the CNN correctly predicted

# First, we'll have the model make predictions on the test image
predicted_classes = initial_cnn.predict(test_X)

# However, each prediction contains 10 probabilities - one probability per class
# The neural network selects the prediction with the highest probability
print("Prediction for the first image: ")
predicted_classes[0]

In [ ]:
# Argmax will select the class with the highest probability
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
print("Predicted class for the first image: ")
predicted_classes[0]

### 6.3 | Display Correct Predictions

In [ ]:
# Identifies the correct images
correct = np.where(predicted_classes==test_Y)[0]
print("Found %d correct labels" % len(correct))

# Displays the first nine via MatPlotLib
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_Y[correct]))
    plt.tight_layout()

### 6.4 | Display Incorrect Predictions

In [ ]:
incorrect = np.where(predicted_classes!=test_Y)[0]
print("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], test_Y[incorrect]))
    plt.tight_layout()

### 6.5 | Classification Report

Now that we have the predicted classes nicely formatted, we can get evaluation metrics in detail. The names of the clothes that correspond to each class ID were https://www.kaggle.com/datasets/zalando-research/fashionmnist.

In [ ]:
target_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

You can see that the classifier is underperforming for class 6 regarding both precision and recall. For class 0 and class 2, the classifier is lacking precision. Also, for class 4, the classifier is slightly lacking both precision and recall.